## 01. data preprocess

In [ ]:
import pandas as pd
import os

In [112]:
df_anno = pd.read_csv("../data/raw/flickr8k/ExpertAnnotations.txt", header=None, delimiter="\t")
df_anno = df_anno.rename(columns={0:"image", 1:"cand_caption"})
df_anno.head(5)

,image,cand_caption,2,3,4
0,1056338697_4f7d7ce270.jpg,2549968784_39bfbe44f9.jpg#2,1,1,1
1,1056338697_4f7d7ce270.jpg,2718495608_d8533e3ac5.jpg#2,1,1,2
2,1056338697_4f7d7ce270.jpg,3181701312_70a379ab6e.jpg#2,1,1,2
3,1056338697_4f7d7ce270.jpg,3207358897_bfa61fa3c6.jpg#2,1,2,2
4,1056338697_4f7d7ce270.jpg,3286822339_5535af6b93.jpg#2,1,1,2


In [113]:
df_cap = pd.read_csv("../data/raw/flickr8k/Flickr8k.token.txt", header=None, delimiter="\t")
df_cap = df_cap.rename(columns={0:"image", 1:"cand_caption"})
df_cap.head(5)

,image,cand_caption
0,1000268201_693b08cb0e.jpg#0,A child in a pink dress is climbing up a set o...
1,1000268201_693b08cb0e.jpg#1,A girl going into a wooden building .
2,1000268201_693b08cb0e.jpg#2,A little girl climbing into a wooden playhouse .
3,1000268201_693b08cb0e.jpg#3,A little girl climbing the stairs to her playh...
4,1000268201_693b08cb0e.jpg#4,A little girl in a pink dress going into a woo...


In [63]:
# build look-up dictionary

with open("../data/raw/flickr8k/Flickr8k.token.txt") as f:
    cap = f.readlines()
    cap = [x.strip() for x in cap]
    
cap_dict = {}
for item in cap:
    _id    = item.split("\t")[0].split("#")[0]
    _index = item.split("\t")[0].split("#")[1]
    _text  = item.split("\t")[1]
    
    if _id in cap_dict:
        cap_dict[_id].append(_text)
        
    else:
        cap_dict[_id] = [_text]        

In [86]:
# image 
imgs = df_anno["image"]
base_path = "../data/raw/flickr8k/Flicker8k_Dataset/"

if os.path.isdir("../data/processed/flickr8k") == False:
    os.mkdir("../data/processed/flickr8k")

with open("../data/processed/flickr8k/imgs.txt", "w") as f:
    for item in imgs:
        f.write(base_path + item + "\n")
        
        
# repetition to match 5 ground truth
with open("../data/processed/flickr8k/imgs_rep_5.txt", "w") as f:
    for item in imgs:
        for i in range(5):
            f.write(base_path + item + "\n")

In [87]:
# cand caption
cand = df_anno["cand_caption"]

with open("../data/processed/flickr8k/cand.txt", "w") as f:
    for item in cand:
        _id    = item.split("#")[0]
        _index = int(item.split("#")[1])
        f.write( cap_dict[_id][_index] + "\n")
        
        
# repetition to match 5 ground truth
with open("../data/processed/flickr8k/cand_rep_5.txt", "w") as f:
    for item in cand:
        for i in range(5):
            _id    = item.split("#")[0]
            _index = int(item.split("#")[1])
            f.write( cap_dict[_id][_index] + "\n")

In [82]:
# gt caption
gt = df_anno["image"]

with open("../data/processed/flickr8k/gt.txt", "w") as f:
    for item in gt:
        for _text in cap_dict[item]:
            f.write( _text + "\n")

In [114]:
# annotations

anno_1 = df_anno[2]
anno_2 = df_anno[3]
anno_3 = df_anno[4]
anno_avg = (anno_1 + anno_2 + anno_3)/3

with open("../data/processed/flickr8k/annotations_avg.txt", "w") as f:
    for i in range(len(anno_avg)):
        f.write(str(anno_avg[i]) + "\n")

## 02. compute score

In [1]:
IMAGE_PATH = "../data/processed/flickr8k/imgs_rep_5.txt"         # input
IMAGE_FEATURE = "../data/processed/flickr8k/image_features.pkl"  # output

FEATURE_EXTRACTOR_BATCH_SIZE = 4
DETECTRON_MODEL  = "../data/detection/detectron_model.pth"
DETECTRON_CONFIG = "../data/detection/detectron_config.yaml"

In [2]:
import sys
sys.path.append("../vilbert")
from extract_features_custom import FeatureExtractor

feature_extractor = FeatureExtractor(model_file    = DETECTRON_MODEL,
                                     config_file   = DETECTRON_CONFIG,
                                     batch_size    = FEATURE_EXTRACTOR_BATCH_SIZE,
)


====================== EVALUATION CONFIGURATION ======================
Summary
----------------------------------------------------------------------
 - model_file                         : ../data/detection/detectron_model.pth
 - config_file                        : ../data/detection/detectron_config.yaml
 - batch_size                         : 4
 - num_features                       : 100
 - output_folder                      : 
 - image_dir                          : 
 - feature_name                       : fc6
 - confidence_threshold               : 0
 - background                         : False
 - partition                          : 0


In [ ]:
with open(IMAGE_PATH) as f:
    image_path = f.readlines()

image_path = [x.strip() for x in image_path]
img_features = feature_extractor.extract_features_direct(image_path)

assert len(image_path) == len(img_features)

0it [00:00, ?it/s]

In [ ]:
import pickle
with open(IMAGE_FEATURE, "wb") as f:
    pickle.dump(img_features, f)